In [ ]:
import pandas as pd
import xgboost as xgb

%matplotlib notebook

In [ ]:
pd.set_option('display.max_columns', 300)

In [ ]:
train_full = pd.read_csv('./sources/train.csv')
train_full

In [ ]:
# Trouver toutes les varaibles categorielles

train_cat = train_full.select_dtypes(include="object")
train_cat

Nous choisissons la feature "Neighborhood" en sus de la variable "Surface" <br />
On crée la variable "Surface":

In [ ]:
train_full['Surface'] = train_full['TotalBsmtSF'] + train_full['GrLivArea']

In [ ]:
train_full['Surface']

On élimine les lignes avec des valeurs manquantes pour "Neighborhood"

In [ ]:
train_full = train_full.dropna(subset=['Neighborhood'])
train_full

In [ ]:
# On plot le prix en fonction de la surface totale en colorant par "Neighborhood"

import seaborn as sns

In [ ]:
sns.scatterplot(x=train_full['Surface'],
                y=train_full['SalePrice'], hue=train_full['Neighborhood'])

Suite au scatterplot, on choisi de supprimer les données dont la surface est supérieure à 7000

In [ ]:
kept_surface = train_full['Surface'] < 7000
train_full = train_full[kept_surface]



Pour intégrer les variables catégorielles, on peut les "séparer" en autant de colones que de catégorie par variables remplies par des `0` si la catégrie ne match pas et `1` si la catégorie match... <br />
fonction `get_dummies` de Pandas

In [ ]:
train_full = pd.get_dummies(train_full, columns=['Neighborhood'])

In [ ]:
train_full

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(
    train_full, train_full['SalePrice'], test_size=0.2, random_state=42)

In [ ]:
neighborhoods = [ col for col in X_train.columns if col.startswith('Neig') ]
X_train = X_train[neighborhoods].join(X_train['SalePrice'])
X_train

On utilise Keras pour entrainer un modèle

In [ ]:
from keras.models import Sequential
from keras.layers import Dense

In [ ]:
model = Sequential([
    Dense(1, input_shape=X_train.shape[1:])
])


from tensorflow.keras.optimizers import SGD
loss ='mse'
LEARNING_RATE = 0.01
model.compile(loss=loss, optimizer=SGD(lr=LEARNING_RATE))
#

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)

#
BATCH_SIZE = X_train.shape[0] # computing the loss over the whole dataset
EPOCHS = 200 # how many iterations over the whole dataset
history = model.fit(X_train, Y_train, epochs=EPOCHS, batch_size=BATCH_SIZE)
#
Y_predict = model.predict(X_train)